## SQL Queries - Simple

### Tasks

For every customer with a 'Main Office' in Dallas show AddressLine1 of the 'Main Office' and AddressLine1 of the 'Shipping' address - if there is no shipping address leave it blank. Use one row per customer.

In [1]:
-- sql here
SELECT CAW.CustomerID, CAST(CompanyName AS CHAR(25)) AS CompanyName, 
       ADR1.AddressLine1 AS 'Main Office addr.',
       COALESCE((SELECT TOP 1 ADR2.AddressLine1 
                 FROM   SalesLT.Address ADR2 
                 JOIN SalesLT.CustomerAddress CA2 
                 ON ADR2.AddressID = CA2.AddressID
                 WHERE CA2.CustomerID = CAW.CustomerID
                   AND CA2.AddressType = 'Shipping'),
                 'use main office addr. ') AS 'Shipping addr.'
FROM   SalesLT.Customer CAW 
  JOIN SalesLT.CustomerAddress CA1 ON CAW.CustomerID = CA1.CustomerID
  JOIN SalesLT.Address ADR1 ON CA1.AddressID = ADR1.AddressID
WHERE City = 'Dallas' AND CA1.AddressType = 'Main Office' 
ORDER BY CAW.CustomerID

(5 rows affected)

Total execution time: 00:00:00.041

CustomerID,CompanyName,Main Office addr.,Shipping addr.
29553,Unsurpassed Bikes,Po Box 8035996,use main office addr.
29574,Third Bike Store,2500 North Stemmons Freeway,use main office addr.
29866,Rental Bikes,"99828 Routh Street, Suite 825",use main office addr.
29948,Town Industries,P.O. Box 6256916,use main office addr.
29954,Elite Bikes,Po Box 8259024,9178 Jumping St.


For each order show the SalesOrderID and SubTotal calculated three ways:

* From the SalesOrderHeader
* Sum of OrderQty * UnitPrice
* Sum of OrderQty * ListPrice

In [7]:
-- sql here
SELECT TOP 3
  SalesOrderHeader.SalesOrderID,
  SalesOrderHeader.SubTotal,
  SUM(SalesOrderDetail.OrderQty * SalesOrderDetail.UnitPrice),
  SUM(SalesOrderDetail.OrderQty * Product.ListPrice)
FROM
  SalesLT.SalesOrderHeader
  JOIN
    SalesLT.SalesOrderDetail
    ON SalesOrderHeader.SalesOrderID = SalesOrderDetail.SalesOrderID
  JOIN
    SalesLT.Product
    ON SalesOrderDetail.ProductID = Product.ProductID
GROUP BY
  SalesOrderHeader.SalesOrderID,
  SalesOrderHeader.SubTotal

(3 rows affected)

Total execution time: 00:00:00.031

SalesOrderID,SubTotal,(No column name),(No column name)
71774,880.3484,713.7960,1189.6600
71776,78.8100,63.9000,106.5000
71780,38418.6895,30600.9960,56651.5600


Show the best selling item by value.

In [5]:
-- sql here
SELECT TOP 3
  Product.Name,
  SUM(SalesOrderDetail.OrderQty * SalesOrderDetail.UnitPrice) AS Total_Sale_Value
FROM
  SalesLT.Product
  JOIN
    SalesLT.SalesOrderDetail
    ON Product.ProductID = SalesOrderDetail.ProductID
GROUP BY
  Product.Name
ORDER BY
  Total_Sale_Value DESC

(3 rows affected)

Total execution time: 00:00:00.024

Name,Total_Sale_Value
"Road-350-W Yellow, 48",38612.4730
"Touring-1000 Blue, 60",37191.4920
"Mountain-200 Black, 42",37178.8380


Show how many orders are in the following ranges (in $):

```
  RANGE      Num Orders      Total Value
    0-  99
  100- 999
 1000-9999
10000-
```

In [3]:
-- sql here
SELECT
  t.range AS 'RANGE',
  COUNT(t.Total) AS 'Num Orders',
  SUM(t.Total) AS 'Total Value'
FROM
  (
    SELECT
    CASE
      WHEN
        SalesOrderDetail.UnitPrice * SalesOrderDetail.OrderQty BETWEEN 0 AND 99
      THEN
        '0-99'
      WHEN
        SalesOrderDetail.UnitPrice * SalesOrderDetail.OrderQty BETWEEN 100 AND 999
      THEN
        '100-999'
      WHEN
        SalesOrderDetail.UnitPrice * SalesOrderDetail.OrderQty BETWEEN 1000 AND 9999
      THEN
        '1000-9999'
      WHEN
        SalesOrderDetail.UnitPrice * SalesOrderDetail.OrderQty > 10000
      THEN
        '10000-'
      ELSE
        'Error'
    END AS 'Range',
    SalesOrderDetail.UnitPrice * SalesOrderDetail.OrderQty AS Total
  FROM
    SalesLT.SalesOrderDetail
  ) t
GROUP BY
  t.range

(4 rows affected)

Total execution time: 00:00:00.029

RANGE,Num Orders,Total Value
0-99,143,7206.4647
100-999,237,94798.8651
1000-9999,153,488074.3050
10000-,9,123923.2788


Identify the three most important cities. Show the break down of top level product category against city.

In [2]:
-- sql here
WITH cte1 AS ( -- collecting basic data
    SELECT ADR.City, PC.Name, SUM(OrderQty*UnitPrice) AS SaleSum
    FROM   SalesLT.ProductCategory PC
      JOIN SalesLT.Product PAW ON PC.ProductCategoryID = PAW.ProductCategoryID
      JOIN SalesLT.SalesOrderDetail SOD ON SOD.ProductID = PAW.ProductID
      JOIN SalesLT.SalesOrderHeader SOH ON SOD.SalesOrderID = SOH.SalesOrderID
      JOIN SalesLT.Customer CAW ON SOH.CustomerID = CAW.CustomerID
      JOIN SalesLT.CustomerAddress CA ON CAW.CustomerID = CA.CustomerID
      JOIN SalesLT.Address ADR ON CA.AddressID = ADR.AddressID
    --WHERE  CA.AddressType = 'Shipping'
    GROUP BY City, PC.Name  ),
  cte2 AS ( -- three most important cities
    SELECT TOP 3 City, SaleSum
    FROM cte1
    ORDER BY SaleSum
  ),
  cte3 AS ( -- max product category per city
    SELECT City, MAX(Salesum) as maxSaleSum
    FROM cte1
    GROUP BY City
  )
 SELECT cte3.City, Name AS ProductCat, maxSaleSum
 FROM cte2 JOIN cte3 ON cte2.City = cte3.City
           JOIN cte1 ON cte2.City = cte1.City
 WHERE cte1.SaleSum = maxSaleSum

(3 rows affected)

Total execution time: 00:00:00.059

City,ProductCat,maxSaleSum
Santa Fe,Touring Bikes,11188.2780
London,Road Bikes,70597.2840
Alhambra,Socks,94.5748


### Notes & Conclusion

Notes are here.